In [1]:
import operator
import re
import sys

In [2]:
from movie import Movie

In [4]:
episode_re = re.compile("\(\d\d\d\d\)\s\{") # FIX: this does not match if the year has nondigit characters
year_re = re.compile("\(\d\d\d\d")

In [52]:
# RATINGS FILE
file_path = "../../imdb/ratings.list"
with open(file_path, 'r') as f:
    l = None
    while(l != "MOVIE RATINGS REPORT"):
        l = f.readline().strip()
    #print(l)
    # empty line
    f.readline()
    # header
    f.readline()
    # Movies
    movies = []
    for m in f:
    #for i in range(100):
    #    m = f.readline()
        if m == "\n":
            break
        m=m.strip()
        try:
            m_list=m.split()
            rating_distribution = m_list[0]
            num_ratings = int(m_list[1])
            mean_rating = float(m_list[2])
            rest = " " . join(m_list[3:])
            if episode_re.search(rest) is None: # if not episode                
                year_match = year_re.search(rest)
                title = rest[:year_match.start()-1]
                year = int(year_match.group()[1:])
                movies.append(Movie(title, year, rating_distribution, num_ratings, mean_rating))
        except:
            #e = sys.exc_info()[0]
            #print( "<p>Error: %s</p>" % e )
            print("Failed to parse movie %s " %m)

Failed to parse movie 1......0.7      12   8.3  "By Any Means" (????) 
Failed to parse movie 1..1221..1       8   5.5  "Das große RTL Special zum Film" (????) {Evan Allmächtig} 
Failed to parse movie 0...0.0.06      12   8.5  "Das große RTL Special zum Film" (????) {Harry Potter und der Orden des Phönix} 
Failed to parse movie 200.0000.4      36   6.3  "Das große RTL Special zum Film" (????) {Harry Potter und die Kammer des Schreckens} 
Failed to parse movie 1....5.3..       6   5.8  "Germside" (????) 
Failed to parse movie ......46..       5   7.6  "Germspesyal" (????) 
Failed to parse movie ......22.6       5   9.0  "TV Rijnmond sport" (????) {Huldiging Feyenoord KNVB Beker 2008} 
Failed to parse movie 0000000124    1201   8.9  Baiohazâdo 7 (????) (VG) 
Failed to parse movie 2........8       5   8.2  Cumming of Age Volume 1 (????) (V) 
Failed to parse movie ....2....8       5   9.0  Faust Fucker - Dr. Fist und seine heißen Stuten (????) (V) 
Failed to parse movie 4...0.00.4      43  

In [104]:
# GENRES FILE
file_path = "../../imdb/genres.list"
with open(file_path, 'r') as f:
    l = None
    while(l != "8: THE GENRES LIST"):
        l = f.readline().strip()
    #print(l)    
    # Dashes
    f.readline()
    # Empty
    f.readline()
    # Movies
    genres = {}
    for m in f:
        #break
    #for i in range(1000):
        m = f.readline()
        if m == "\n":
            break
        m=m.strip()
        try:
            m_list=m.split()
            #print(m_list)
            genre = m_list[-1]            
            if episode_re.search(m) is None: # if not episode                
                year_match = year_re.search(m)
                title = m[:year_match.start()-1]
                year = int(year_match.group()[1:])
                #movies.append(Movie(title, year, rating_distribution, num_ratings, mean_rating))
                #print("%s \t %d \t %s" %(title, year, genre))
                if title in genres: # FIX: TWO OR MORE TITLES IN SEPARATE YEARS, GETS MATCHED TO THE SAME GENRE
                    genres[title].append(genre)
                else:
                    genres[title]=[year, genre]
        except:
            pass
            #e = sys.exc_info()[0]
            #print( "<p>Error: %s</p>" % e )
            #print("Failed to parse movie %s " %m)

In [105]:
len(movies)

447074

In [106]:
len(genres)

862113

In [109]:
count=0
for m in movies:
    if m.title not in genres:
        count += 1
    else:
        genre = genres[m.title]
        if m.year == int(genre[0]):
            m.genre = genre[1:]
count

98839

In [110]:
m_ = movies
#m_=list(filter(lambda m: m.year>2000, m_))
m_=list(filter(lambda m: m.num_ratings>10000, m_))
m_=list(filter(lambda m: m.mean_rating>9, m_))

In [111]:
len(m_)

54

In [112]:
m_=sorted(m_, key=operator.attrgetter('num_ratings'), reverse=True)
#m_=sorted(m_, key=operator.attrgetter('year'), reverse=True)
#m_=sorted(m_, key=operator.attrgetter('mean_rating'), reverse=True)

In [113]:
m_[:20]

[The Shawshank Redemption	1994	0000000125	1812040	9.3,
 The Godfather	1972	0000000125	1237499	9.2,
 "Game of Thrones"	2011	0000000016	1154888	9.5,
 "Breaking Bad"	2008	0000000016	982500	9.5,
 "Sherlock"	2010	0000000025	577762	9.2,
 "True Detective"	2014	0000000134	358781	9.1,
 "Band of Brothers"	2001	0000000016	258283	9.5,
 "The Sopranos"	1999	0000000125	208571	9.2,
 "The Wire"	2002	0000000016	207354	9.3,
 "Firefly"	2002	0000000125	200906	9.1,
 "Avatar: The Last Airbender"	2005	0000000025	148275	9.2,
 "Rick and Morty"	2013	0000000026	122929	9.3,
 "Planet Earth"	2006	0000000016	118241	9.5,
 Dag II	2016	0000000009	88187	9.8,
 "Leyla ile Mecnun"	2011	0000000008	75114	9.2,
 "Cosmos: A Spacetime Odyssey"	2014	0000000026	71628	9.3,
 "Hagane no renkinjutsushi"	2009	0000000125	60599	9.1,
 "Last Week Tonight with John Oliver"	2014	0000000124	48189	9.1,
 "One Punch Man: Wanpanman"	2015	0000000125	45398	9.1,
 CM101MMXI Fundamentals	2013	0000000017	37222	9.3]

In [114]:
m_[0]

The Shawshank Redemption	1994	0000000125	1812040	9.3

In [115]:
m_[0].genre

['Drama']

In [116]:
genres[m_[0].title]

[1994, 'Drama']

In [118]:
for m in m_:
   print(m.genre)

['Drama']


AttributeError: 'Movie' object has no attribute 'genre'

In [119]:
m_[1]

The Godfather	1972	0000000125	1237499	9.2

In [120]:
genres[m_[1].title]

[1911, 'Drama', 'Short', 'Drama', 'Crime', 'Crime', 'Thriller', 'Short']